In [31]:
import os.path

from shared_code.utility.schemas.spark_table_schema import image_table_schema
from shared_code.utility.spark.set_environ import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
set_azure_env()

spark = SparkSession \
	.builder \
	.appName('add-missing-captions') \
	.master("local[6]") \
	.config("spark.cores.max", "1") \
	.config("spark.executor.instances", "1") \
	.config("spark.executor.cores", "1") \
	.config("spark.executor.cores", "1") \
	.config("spark.executor.instances", "1") \
	.config("spark.driver.memory", "10g") \
	.config("spark.memory.offHeap.enabled", True) \
	.config("spark.memory.offHeap.size", "10g") \
	.config("spark.executor.cores", "1")

spark_context = spark.getOrCreate()

In [32]:
def get_vit_caption():
	model = VisionEncoderDecoderModel.from_pretrained("D:\\models\\vit-gpt2-image-captioning")
	feature_extractor = ViTFeatureExtractor.from_pretrained("D:\\models\\vit-gpt2-image-captioning")
	tokenizer = AutoTokenizer.from_pretrained("D:\\models\\vit-gpt2-image-captioning")
	return model, feature_extractor, tokenizer

def get_default_caption():
	model = VisionEncoderDecoderModel.from_pretrained("D:\\models\\image-caption-generator")
	feature_extractor = ViTFeatureExtractor.from_pretrained("D:\\models\\image-caption-generator")
	tokenizer = GPT2TokenizerFast.from_pretrained("D:\\models\\image-caption-generator\\tokenizer")
	return model, feature_extractor, tokenizer

In [33]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, GPT2TokenizerFast,AutoTokenizer
from PIL import Image

model, feature_extractor, tokenizer = get_vit_caption()

def caption_image_vit(image_path: str) -> str:
	try:
		max_length = 32

		num_beams = 4

		gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

		images = []

		i_image = Image.open(image_path)
		if i_image.mode != "RGB":
			i_image = i_image.convert(mode="RGB")

		images.append(i_image)

		print(f":: Predicting image: {image_path}")

		pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values

		output_ids = model.generate(pixel_values, **gen_kwargs)

		print(f":: Decoding output for image: {image_path}")
		predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

		prediction = [prediction.strip() for prediction in predictions]

		print(f":: Completed prediction for image: {image_path}")
		if len(prediction) > 0:
			return prediction[0]
		else:
			return None

	except Exception as e:
		print(f":: Process Failed For {image_path} with {e}")
		return None

In [34]:
spark_df = spark_context.read.parquet("D:\\data\\processed\\reddit_images_processed.parquet", schema=image_table_schema)

print("== Data Loaded From Parquet ==")
display(spark_df.limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11094it,D:\data\images\greentext\q0rxrphnuqha1.jpg,Anon is insulinpilled,11094it,TheGourmetGaylord,https://i.redd.it/q0rxrphnuqha1.jpg,None,/r/greentext/comments/11094it/anon_is_insulinp...,d39377ac12d266df304429d5b03ff483,greentext,None,True,q0rxrphnuqha1.jpg,None,D:\data\images\greentext\thumbnail\q0rxrphnuqh...,False
1,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,a double decker bus is driving down the street,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
2,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,a woman in a bikini standing next to a tree,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
3,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,a cityscape of a city with tall buildings,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
4,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,a woman with glasses and a cat in her lap,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
5,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,a large building with a clock on top of it,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
6,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,a woman holding a cell phone in front of her face,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
7,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,a woman taking a picture of herself in a bathr...,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
8,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,a woman holding a bottle of beer next to a wall,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False
9,training,110c9o8,D:\data\images\CityPorn\ndc1hr2wvrha1.jpg,"Quetta, the Tucson of Pakistan",110c9

In [35]:
from shared_code.utility.scripts.image_analysis import ImageCaption
from shared_code.utility.storage.table import TableAdapter

table_adapter: TableAdapter = TableAdapter()
caption_instance = ImageCaption()

def get_caption_image_name(image_path: str, caption: str, row_key: str, partition_key: str):
	if not os.path.exists(image_path):
		return ""
	try:
		if caption is None or caption == "" or caption == "NaN" or caption == "None":
			caption = caption_image_vit(image_path)
			record = table_adapter.get_entity("training", partition_key, row_key)
			record["updated_caption"] = caption
			table_adapter.upsert_entity_to_table("training", record)
			return caption
		else:
			return caption
	except:
		return ""

In [36]:
temp_df = spark_context.createDataFrame(list(table_adapter.get_all_entities("training")), schema=image_table_schema)

temp_df.createOrReplaceTempView("temp_df")

remaining_captions = temp_df\
	.where("updated_caption == ''") \
	.where("updated_caption == 'NaN'")\
	.select("small_image", "updated_caption", "RowKey", "PartitionKey")\
	.collect()

remaining_captions_for_vit = spark_context.sql("""
	select * from temp_df
		where
		(updated_caption == '' or updated_caption == 'NaN' or updated_caption == 'None')
		and exists == True
""")


print("== Data Loaded For Captions ==")
print(f"Need to processes: {remaining_captions_for_vit.count()} image captions")
display(remaining_captions_for_vit.limit(100).toPandas())

remaining_count = remaining_captions_for_vit.count()

Need to processes: 148 image captions


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,113vpeh,D:\data\images\SFWRedheads\9wp2pwmokmia1.jpg,Renzi at MetArt,113vpeh,AbbreviationsTop3003,https://i.redd.it/9wp2pwmokmia1.jpg,NaN,/r/SFWRedheads/comments/113vpeh/renzi_at_metart/,95b9239feb27095d5fe5500162efa644,SFWRedheads,A woman in a red dress is sitting on a stone b...,True,9wp2pwmokmia1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\9wp2pwmok...,False
1,training,113vvqb,D:\data\images\SFWRedheads\zdmtllq0mmia1.jpg,what’s the best/worst advice you’ve ever been ...,113vvqb,tantric_eden_,https://i.redd.it/zdmtllq0mmia1.jpg,NaN,/r/SFWRedheads/comments/113vvqb/whats_the_best...,14568464469e726c3b5afd9704f9722e,SFWRedheads,A girl in a pink shirt is swinging on a swing ...,True,zdmtllq0mmia1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\zdmtllq0m...,False
2,training,113vwnm,D:\data\images\SFWRedheads\tsc693l7mmia1.jpg,I’d sit next to me at lunch.,113vwnm,m0llyAna,https://i.redd.it/tsc693l7mmia1.jpg,NaN,/r/SFWRedheads/comments/113vwnm/id_sit_next_to...,d0843e5d3ac0bfb61d1dcaefc1710695,SFWRedheads,A little girl in a pink dress is sitting on a ...,True,tsc693l7mmia1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\tsc693l7m...,False
3,training,113vxs4,D:\data\images\CityPorn\r9fenert4lia1.jpg,"Musashi-Kosugi, Kanagawa, Japan [OC] [#4688]",113vxs4,tokyo_form,https://i.redd.it/r9fenert4lia1.jpg,NaN,/r/CityPorn/comments/113vxs4/musashikosugi_kan...,ed4df16c56be363a43ee472b08f7a5f9,CityPorn,A person is flying through the air over the oc...,True,r9fenert4lia1.jpg,NaN,D:\data\images\CityPorn\thumbnail\r9fenert4lia...,False
4,training,113w0wv,D:\data\images\EarthPorn\rsjmwpz3nmia1.jpg,"The island I’m from. Moloka’i, Hawai’i [1824X1...",113w0wv,iamayeshaerotica,https://i.redd.it/rsjmwpz3nmia1.jpg,NaN,/r/EarthPorn/comments/113w0wv/the_island_im_fr...,6943599bd7be7477de99725e6d5e001d,EarthPorn,A group of people hike up a mountain.,True,rsjmwpz3nmia1.jpg,NaN,D:\data\images\EarthPorn\thumbnail\rsjmwpz3nmi...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,training,ymb965,D:\data\images\sfwpetite\2ii7y48b70y91.jpg,Elevate,ymb965,sandycheeseit,https://i.redd.it/2ii7y48b70y91.jpg,NaN,/r/sfwpetite/comments/ymb965/elevate/,26bed84f6b0c08cfa7585cd9c70fa9e3,sfwpetite,A little girl in a pink shirt and blue jeans i...,True,2ii7y48b70y91.jpg,NaN,D:\data\images\sfwpetite\thumbnail\2ii7y48b70y...,False
96,training,ymczwr,D:\data\images\sfwpetite\l125jbihk0y91.jpg,Barbie,ymczwr,Live_Accident_8812,https://i.redd.it/l125jbihk0y91.jpg,NaN,/r/sfwpetite/comments/ymczwr/barbie/,ed7de5f9d80b725c5c1f978dce826093,sfwpetite,A little girl in a pink dress is sitting on a ...,True,l125jbihk0y91.jpg,NaN,D:\data\images\sfwpetite\thumbnail\l125jbihk0y...,False
97,training,ymhhka,D:\data\images\Amicute\2NtFXGw.jpg,If I can still walk - then we are not finished 🥱,ymhhka,AdemoNarmcremhr,https://i.imgur.com/2NtFXGw.jpg,Female (18+),/r/Amicute/comments/ymhhka/if_i_can_still_walk...,3e162537513cf4927525e5a0fc37f35a,Amicute,A girl in a pink dress is sitting on a red couch.,True,2NtFXGw.jpg,NaN,D:\data\images\Amicute\thumbnail\2NtFXGw.jpg,False
98,training,yml4xo,D:\data\images\AmIhotAF\45eqvaF.jpg,My Minnie Me (35),yml4xo,AboutTheBoobs,https://i.imgur.com/45eqvaF.jpg,NaN,/r/AmIhotAF/comments/yml4xo/my_minnie_me_35/,5a6d8460390ad8f8b47e6ca389c1d6e6,AmIhotAF,A little girl in a pink dress is sitting on a ...,True,45eqvaF.jpg,NaN,D:\data\images\AmIhotAF\thumbnail\45eqvaF.jpg,False


In [37]:
for elem in remaining_captions_for_vit.collect():
	try:
		if remaining_count % 100 == 0:
			print(f"Remaining: {remaining_count}")
		if os.path.exists(elem.small_image):
			entity = table_adapter.get_entity("training", elem.PartitionKey, elem.RowKey)
			try:
				result = caption_image_vit(elem.small_image)
			except:
				result = None
			entity["updated_caption"] = result
			table_adapter.upsert_entity_to_table("training", entity)
		else:
			print("File not found: ", elem.small_image)
			entity = table_adapter.get_entity("training", elem.PartitionKey, elem.RowKey)
			entity['exists'] = False
			table_adapter.upsert_entity_to_table("training", entity)
	except ConnectionResetError:
		print("Connection Reset Error")
		table_adapter = TableAdapter()
		continue

:: Predicting image: D:\data\images\SFWRedheads\thumbnail\9wp2pwmokmia1.jpg
:: Decoding output for image: D:\data\images\SFWRedheads\thumbnail\9wp2pwmokmia1.jpg
:: Completed prediction for image: D:\data\images\SFWRedheads\thumbnail\9wp2pwmokmia1.jpg
:: Predicting image: D:\data\images\SFWRedheads\thumbnail\zdmtllq0mmia1.jpg
:: Decoding output for image: D:\data\images\SFWRedheads\thumbnail\zdmtllq0mmia1.jpg
:: Completed prediction for image: D:\data\images\SFWRedheads\thumbnail\zdmtllq0mmia1.jpg
:: Predicting image: D:\data\images\SFWRedheads\thumbnail\tsc693l7mmia1.jpg
:: Decoding output for image: D:\data\images\SFWRedheads\thumbnail\tsc693l7mmia1.jpg
:: Completed prediction for image: D:\data\images\SFWRedheads\thumbnail\tsc693l7mmia1.jpg
:: Predicting image: D:\data\images\CityPorn\thumbnail\r9fenert4lia1.jpg
:: Decoding output for image: D:\data\images\CityPorn\thumbnail\r9fenert4lia1.jpg
:: Completed prediction for image: D:\data\images\CityPorn\thumbnail\r9fenert4lia1.jpg
:: Pr

In [38]:
refreshed = spark_context.createDataFrame(list(table_adapter.get_all_entities("training")), schema=image_table_schema)

display(refreshed.limit(10).toPandas())

refreshed.write.parquet("D:\\data\\processed\\reddit_images_processed_vit_caption.parquet", mode="overwrite")

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,New York in the fog,1000cej,OtterlyFoxy,https://i.redd.it/4emw5uldib9a1.jpg,NaN,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,7a8d96e378c15c8ab8440ac311f12c11,CityPorn,A person is riding a bicycle on a city street.,True,4emw5uldib9a1.jpg,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,False
1,training,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,Thoughts about my NYE party outfit?,1000d16,princessxo699,https://i.imgur.com/GgFEagO.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,9951b4f82caeb8ba2bd9f79f8d422450,SFWNextDoorGirls,A little girl in a pink dress is holding a pin...,True,GgFEagO.jpg,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,False
2,training,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,(IKTR),1000fg0,BlkBrd1312,https://i.redd.it/nwa7hts2jb9a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/1000fg0/iktr/,be8dd55e34216bec1e15e03fa296eacc,HotGirlNextDoor,A woman in a pink bathing suit holds a baby in...,True,nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,False
3,training,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,Just looking for entertainment,1000glf,toolate_sharkbait,https://i.redd.it/4xyb1vgbjb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1000glf/just_looking_for_...,e554c1ed7ffa2740436ac082068b2824,AmIhotAF,A woman in a pink dress is smiling at the camera.,True,4xyb1vgbjb9a1.jpg,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,False
4,training,1000j1n,/data/images/greentext/3mewbe0wjb9a1.jpg,Anon wants Elon cut,1000j1n,trent8051,https://i.redd.it/3mewbe0wjb9a1.jpg,None,/r/greentext/comments/1000j1n/anon_wants_elon_...,1dec3dabb5e46cde01855d06089c287a,greentext,A man and a woman are sitting at a table with ...,True,3mewbe0wjb9a1.jpg,a collage of photos showing a man in a suit an...,/data/images/greentext/thumbnail/3mewbe0wjb9a1...,False
5,training,1000qiq,D:\data\images\SFWRedheads\0mgrt95plb9a1.jpg,Who’s going to be my new years kiss tonight? 🤧,1000qiq,fireybubbulegum,https://i.redd.it/0mgrt95plb9a1.jpg,NaN,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,7b0be38a08e3316051c6e5a3753bc791,SFWRedheads,A woman in a red dress is talking to a man in ...,False,0mgrt95plb9a1.jpg,NaN,None,False
6,training,1001fon,D:\data\images\SFWRedheads\gp4fhsdorb9a1.jpg,honk if you like my sweater 😁💕,1001fon,glam_pire,https://i.redd.it/gp4fhsdorb9a1.jpg,NaN,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,feed678c3b74ed1d0b7011f28e8f283f,SFWRedheads,A woman in a red dress is talking to a man in ...,False,gp4fhsdorb9a1.jpg,NaN,None,False
7,training,1001ghp,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,Braids,1001ghp,eromorphic,https://i.redd.it/v3ra9g4vrb9a1.jpg,NaN,/r/sfwpetite/comments/1001ghp/braids/,16ca1536c4f6936d3e994df7cb40ab8e,sfwpetite,A girl in a pink bathing suit walks down the b...,True,v3ra9g4vrb9a1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,False
8,training,1001ih5,D:\data\images\SFWNextDoorGirls\bhigr5rbsb9a1.jpg,New Year's Elf,1001ih5,ms_twine_grace,https://i.redd.it/bhigr5rbsb9a1.jpg,NaN,/r/SFWNextDoorGirls/comments/1001ih5/new_years...,9ff18e9afb7b566c0d50add45cbfb989,SFWNextDoorGirls,A little girl in a pink dress is dancing on a ...,False,bhigr5rbsb9a1.jpg,NaN,None,False
9,training,1001kp4,D:\data\images\AmIhotAF\dn7lcclvsb9a1.jpg,Happy New Years Eve,1001kp4,Hotblondie69420,https://i.redd.it/dn7lcclvsb9a1.jpg,Female (18+),/r/AmIhotAF/comments/1001kp4/happy_new_years_eve/,7137ef4fe19a97c757553bfa62b23a33,AmIhotAF,A woman in a white dress is talking to a man i...,False,dn7lcclvsb9a1.jpg,NaN,N

In [39]:
df = spark_context.read.parquet("D:\\data\\processed\\reddit_images_processed_vit_caption.parquet", schema=image_table_schema)

print("== Data Loaded From Parquet ==")
display(df.limit(10).toPandas())

== Data Loaded From Parquet ==


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,10nlv1e,/data/images/greentext/oq8xv2bgcvea1.jpg,Anon at a family reunion,10nlv1e,Kwispy92,https://i.redd.it/oq8xv2bgcvea1.jpg,None,/r/greentext/comments/10nlv1e/anon_at_a_family...,3e3beee2d82def97e9d753f2b3de7398,greentext,None,True,oq8xv2bgcvea1.jpg,A man in a red shirt and blue jeans is sitting...,/data/images/greentext/thumbnail/oq8xv2bgcvea1...,False
1,training,10nm2yp,D:\data\images\SFWRedheads\rhmaiya3evea1.jpg,In my favorite outfit ✨ what’s yours?,10nm2yp,davine__,https://i.redd.it/rhmaiya3evea1.jpg,NaN,/r/SFWRedheads/comments/10nm2yp/in_my_favorite...,85dad0fe57d2965d9c6091d914b47e3f,SFWRedheads,A little girl in a pink dress is sitting on a ...,False,rhmaiya3evea1.jpg,NaN,None,False
2,training,10nm55b,D:\data\images\sfwpetite\o2xorznywtea1.jpg,Another of the girl from yesterday...,10nm55b,insecure_walrus,https://i.redd.it/o2xorznywtea1.jpg,NaN,/r/sfwpetite/comments/10nm55b/another_of_the_g...,4e60a999943248a425b869eeda650581,sfwpetite,A woman in a bikini and a man in a white tank ...,True,o2xorznywtea1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\o2xorznywte...,False
3,training,10nmlp1,D:\data\images\HotGirlNextDoor\ZBeK9Ra.jpg,Bikini Babes (IKTR),10nmlp1,figurephilosopher,https://i.imgur.com/ZBeK9Ra.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/10nmlp1/bikini_bab...,b98400d9569192d3560e7fae92cefba0,HotGirlNextDoor,A woman in a bikini and a man in a white t-shi...,True,ZBeK9Ra.jpg,two women sitting on top of a sandy beach,D:\data\images\HotGirlNextDoor\thumbnail\ZBeK9...,False
4,training,10nms0d,D:\data\images\CityPorn\mykqut7cjvea1.jpg,Central Park with some fog,10nms0d,silent-skreams,https://i.redd.it/mykqut7cjvea1.jpg,NaN,/r/CityPorn/comments/10nms0d/central_park_with...,8b6caf5a265892152a6af7523a85b1dc,CityPorn,A person in a red jacket is skiing down a snow...,True,mykqut7cjvea1.jpg,a city skyline with a plane flying over it,D:\data\images\CityPorn\thumbnail\mykqut7cjvea...,False
5,training,10nmw96,D:\data\images\CityPorn\d4pxa1d8kvea1.jpg,"Parque Piqueri @ São Paulo, Brazil.",10nmw96,Electrical_Eye_7427,https://i.redd.it/d4pxa1d8kvea1.jpg,NaN,/r/CityPorn/comments/10nmw96/parque_piqueri_sã...,922c54cf1bdd32e76a85973975d8b446,CityPorn,A man in a blue shirt and khaki pants is stand...,True,d4pxa1d8kvea1.jpg,a park filled with trees and shrubs in the mid...,D:\data\images\CityPorn\thumbnail\d4pxa1d8kvea...,False
6,training,10nnaq1,D:\data\images\HotGirlNextDoor\gdfpb17o5uea1.jpg,Burnt Cheeks 😝(IKTR),10nnaq1,insecure_walrus,https://i.redd.it/gdfpb17o5uea1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/10nnaq1/burnt_chee...,ce3cb85a2d06d7260c2e310c1f188a42,HotGirlNextDoor,A little girl in a pink bathing suit is standi...,False,gdfpb17o5uea1.jpg,NaN,None,False
7,training,10nng2e,/data/images/greentext/58i3hv7covea1.jpg,Anon’s special order,10nng2e,KayJay65,https://i.redd.it/58i3hv7covea1.jpg,None,/r/greentext/comments/10nng2e/anons_special_or...,46b6c05ef050a277623a73a716aa84ff,greentext,None,True,58i3hv7covea1.jpg,None,/data/images/greentext/thumbnail/58i3hv7covea1...,False
8,training,10nnrvp,D:\data\images\CityPorn\rn8dtsftqvea1.jpg,"Dublin, Ireland",10nnrvp,user89135,https://i.redd.it/rn8dtsftqvea1.jpg,NaN,/r/CityPorn/comments/10nnrvp/dublin_ireland/,08d9d61765fec72212d0128fab8ec6ac,CityPorn,A group of people walk down the street.,True,rn8dtsftqvea1.jpg,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\rn8dtsftqvea...,False
9,training,10no4io,D:\data\images\SFWRedheads\jNYxEhG.jpg,A cute mirror selfie for you &lt;3,10no4io,Yummy_Cummy,https://i.imgur.com/jNYxEhG.jpg,NaN,/r/SFWRedheads/comments/10no4io/a_cute_mirror_...,4982bbf1e49dcac046d7db82913cf6d2,SFWRedheads,A woman in a white dress is sitting on a couch...,True,jNYxEhG.jpg,a woman sitting on a bed looking at her cell p...,D:\data\imag